### Load Eulerian and Lagrangian filtered signals

#### Eulerian :
X = U, V

SSX_rot_hat : Complex band pass filtered signal

SSX_rot_demodulated : Complex envelope of the filtered signal

SSX_rot_demodulated (+_real or_imag) : real and imaginary part of the complex envelope

#### NB : 
The Eulerian datasets for SSU and V do not have the same spatial resolution. SSU has been stored at maximal resolution (4320 points in both i and j direction). SSV has been sub-sampled (1080 points).

### Lagrangian : 
zonal (or merdional) +_velocity_hat (+_real or _imag) : real and imaginary part of the band pass filtered signal

zonal (or merdional) +_velocity_demodulated (+_real or _imag) : real and imaginary part of the complex envelope

In [2]:
import numpy as np
import geopandas as gpd


import xarray as xr
from matplotlib import pyplot as plt
%matplotlib inline

from xhistogram.xarray import histogram
import dask.dataframe as dd
import cartopy.crs as ccrs
import cartopy.feature as cfeature

import mitequinox.utils as ut
from mitequinox.plot import *
import mitequinox.parcels as pa
from xmitgcm import llcreader

from fsspec.implementations.local import LocalFileSystem

In [3]:
from dask.distributed import Client, LocalCluster
#
cluster = LocalCluster()
#
#from dask_jobqueue import PBSCluster
#cluster = PBSCluster() #
#w = cluster.scale(jobs=2
#                )
#
client = Client(cluster)

#### 

In [5]:
#Eulerian dataset

dsU_hf = xr.open_zarr(ut.work_data_dir+'filtered_itide/SSU_bandpassed_filtered.zarr') #SSU, filtered dataset
dsV_hf = xr.open_zarr(ut.work_data_dir+'filtered_itide/SSV_filtered.zarr') #SSU, filtered dataset
#grid = xr.open_zarr(ut.work_data_dir+'rechunked/SSU_rot.zarr')[['XC','YC','Depth']]
dsU_hf

,Array,Chunk
Bytes,30.50 TiB,659.18 MiB
Shape,"(13, 4320, 4320, 8640)","(1, 50, 100, 8640)"
Count,49765 Tasks,49764 Chunks
Type,complex128,numpy.ndarray
,Array,Chunk
Bytes,15.25 TiB,329.59 MiB
Shape,"(13, 4320, 4320, 8640)","(1, 50, 100, 8640)"
Count,49765 Tasks,49764 Chunks
Type,float64,numpy.ndarray
,Array,Chunk


In [6]:
dsV_hf

,Array,Chunk
Bytes,1.91 TiB,1.29 GiB
Shape,"(13, 1080, 1080, 8640)","(1, 100, 100, 8640)"
Count,1574 Tasks,1573 Chunks
Type,complex128,numpy.ndarray
,Array,Chunk
Bytes,0.95 TiB,659.18 MiB
Shape,"(13, 1080, 1080, 8640)","(1, 100, 100, 8640)"
Count,1574 Tasks,1573 Chunks
Type,float64,numpy.ndarray
,Array,Chunk


In [7]:
#Lagrangian dataset
root_dir = '/home/datawork-lops-osi/equinox/mit4320/parcels/'
run_name = 'global_T365j_dt1j_dij50'

parcels_index = 'filtered_itide_Tw30_sd_band04'

# choose to select time indexed data for now
p = pa.parcels_output(root_dir+run_name, parquets=[parcels_index])
df = p[parcels_index]
df.head()

,trajectory,lat,lon,z,zonal_velocity,meridional_velocity,waterdepth,zonal_velocity_hat_real,zonal_velocity_hat_imag,zonal_velocity_demodulated_real,zonal_velocity_demodulated_imag,meridional_velocity_hat_real,meridional_velocity_hat_imag,meridional_velocity_demodulated_real,meridional_velocity_demodulated_imag
time,,,,,,,,,,,,,,,
2011-11-15,2000844,-67.226303,0.562500,0.0,0.0,0.0,0.0,0.002438,0.002383,0.002438,0.002383,-0.001410,0.005584,-0.001410,0.005584
2011-11-15,32001255,-69.530426,-126.770836,0.0,0.0,0.0,0.0,0.000518,-0.006434,0.000518,-0.006434,0.002277,-0.000432,0.002277,-0.000432
2011-11-15,24000949,-40.126877,163.312500,0.0,0.0,0.0,0.0,0.028092,0.000022,0.028092,0.000022,-0.018097,-0.020551,-0.018097,-0.020551
2011-11-15,32001165,-65.565285,-130.937500,0.0,0.0,0.0,0.0,-0.004214,-0.001272,-0.004214,-0.001272,0.000220,-0.000716,0.000220,-0.000716
2011-11-15,24001270,-29.922403,177.895828,0.0,0.0,0.0,0.0,-0.027028,0.013208,-0.027028,0.013208,0.014038,0.009028,0.014038,0.009028


In [8]:
cluster.close()

/home1/datahome/zcaspar/miniconda3/envs/croco/lib/python3.8/contextlib.py:120: UserWarning: Creating scratch directories is taking a surprisingly long time. This is often due to running workers on a network file system. Consider specifying a local-directory to point workers to write scratch data to a local disk.
  next(self.gen)
/home1/datahome/zcaspar/miniconda3/envs/croco/lib/python3.8/contextlib.py:120: UserWarning: Creating scratch directories is taking a surprisingly long time. This is often due to running workers on a network file system. Consider specifying a local-directory to point workers to write scratch data to a local disk.
  next(self.gen)
/home1/datahome/zcaspar/miniconda3/envs/croco/lib/python3.8/contextlib.py:120: UserWarning: Creating scratch directories is taking a surprisingly long time. This is often due to running workers on a network file system. Consider specifying a local-directory to point workers to write scratch data to a local disk.
  next(self.gen)
/home1/